### Goal of the notebook

* The small version of the dataset has around 100K ratings on 9k movies from 700 users
* The full version of the dataset has around 25M ratings on 45k movies from 270k users

* In this notebook, I will try to generate a sub-graph of the full dataset graph, without being biased torwards the sparsity of the new graph
    * I will try filtering the dataset based on the movies (keep only a percentage of movies) and later on keep only the relevant ratings and users
    * I will try filtering based on the users

### Imports

In [5]:
import pathlib
import os
import sys
from collections import defaultdict
from statistics import mean
from py2neo import Graph
from py2neo.bulk import merge_nodes, merge_relationships
import random

parent_path = pathlib.Path(os.getcwd()).parent.absolute()
sys.path.append(str(parent_path))

from utils.general import read_csv, df_to_json
from tqdm import tqdm
from tabulate import tabulate


### Load CSVs

In [6]:
data_dir = "movies_with_metadata"

In [7]:
movies_json = df_to_json(
    read_csv(
        filename="movies_metadata",
        parent_dir_name=data_dir,
        low_memory=False,
    )
)

Reading from: /Users/ioannisathanasiou/diploma/model/movies_with_metadata/movies_metadata.csv


In [8]:
ratings_json = df_to_json(
    read_csv(
        filename="ratings",
        parent_dir_name=data_dir,
    )
)


Reading from: /Users/ioannisathanasiou/diploma/model/movies_with_metadata/ratings.csv


### Movies

In [9]:
movies_json[0]

{'adult': 'False',
 'belongs_to_collection': "{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",
 'budget': '30000000',
 'genres': "[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",
 'homepage': 'http://toystory.disney.com/toy-story',
 'id': '862',
 'imdb_id': 'tt0114709',
 'original_language': 'en',
 'original_title': 'Toy Story',
 'overview': "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
 'popularity': '21.946943',
 'poster_path': '/rhIRbceoE9lR4veEXuwCC2wARtG.jpg',
 'production_companies': "[{'name': 'Pixar Animation Studios', 'id': 3}]",
 'production_countries': "[{'iso_3166_1':

In [10]:
len(movies_json)

45466

#### Ratings

In [11]:
ratings_json[0]

{'userId': 1, 'movieId': 110, 'rating': 1.0, 'timestamp': 1425941529}

In [12]:
len(ratings_json)

26024289

### Filter ratings based on movies

I will find the average ratings per movie, and observe how the size of the dataset changes if I keep only the movies that have more ratings than a specific threshold

In [13]:
rated_movies_ids = set([
    rating["movieId"]
    for rating in ratings_json
])

In [14]:
len(rated_movies_ids)

45115

In [15]:
ratings_per_movie = defaultdict(list)
for rating in ratings_json:
    ratings_per_movie[rating["movieId"]].append(rating)

In [16]:
ratings_per_movie_count = { movieId: len(ratings) for movieId, ratings in ratings_per_movie.items() }

In [17]:
avg_ratings_count = mean(ratings_per_movie_count.values())
avg_ratings_count

576.8433780339134

In [18]:
def count_movies_ratings_by_threshold(multiplier):
    threshold = avg_ratings_count * multiplier
    very_rated_movies = set(
        [movieId for movieId, ratings in ratings_per_movie_count.items() if ratings >= threshold])
    little_rated_movies = set(
        [movieId for movieId, ratings in ratings_per_movie_count.items() if ratings < threshold])
    ratings_on_very_rated_movies = [
        rating for rating in ratings_json if rating["movieId"] in very_rated_movies]
    ratings_on_little_rated_movies = [
        rating for rating in ratings_json if rating["movieId"] in little_rated_movies]
    return [multiplier, threshold, len(very_rated_movies), len(little_rated_movies), len(ratings_on_very_rated_movies), len(ratings_on_little_rated_movies)]


In [15]:
results = []
for i in range(1, 40):
    print("Multipler:", i/10)
    results.append(count_movies_ratings_by_threshold(i/10))
print(tabulate(results, headers=["High Movies", "Low movies", "Ratings on High", "Ratings on Low"], tablefmt="github"))

Multipler: 0.1
Multipler: 0.2
Multipler: 0.3
Multipler: 0.4
Multipler: 0.5
Multipler: 0.6
Multipler: 0.7
Multipler: 0.8
Multipler: 0.9
Multipler: 1.0
Multipler: 1.1
Multipler: 1.2
Multipler: 1.3
Multipler: 1.4
Multipler: 1.5
Multipler: 1.6
Multipler: 1.7
Multipler: 1.8
Multipler: 1.9
Multipler: 2.0
Multipler: 2.1
Multipler: 2.2
Multipler: 2.3
Multipler: 2.4
Multipler: 2.5
Multipler: 2.6
Multipler: 2.7
Multipler: 2.8
Multipler: 2.9
Multipler: 3.0
Multipler: 3.1
Multipler: 3.2
Multipler: 3.3
Multipler: 3.4
Multipler: 3.5
Multipler: 3.6
Multipler: 3.7
Multipler: 3.8
Multipler: 3.9
|     |           |   High Movies |   Low movies |   Ratings on High |   Ratings on Low |
|-----|-----------|---------------|--------------|-------------------|------------------|
| 0.1 |   57.6843 |         11907 |        33208 |          25720244 |           304045 |
| 0.2 |  115.369  |          9489 |        35626 |          25520781 |           503508 |
| 0.3 |  173.053  |          8225 |        36890 |     

In [16]:
results = []
for i in range(40, 60):
    print("Multipler:", i/10)
    results.append(count_movies_ratings_by_threshold(i/10))
print(tabulate(results, headers=["High Movies", "Low movies", "Ratings on High", "Ratings on Low"], tablefmt="github"))

Multipler: 4.0
Multipler: 4.1
Multipler: 4.2
Multipler: 4.3
Multipler: 4.4
Multipler: 4.5
Multipler: 4.6
Multipler: 4.7
Multipler: 4.8
Multipler: 4.9
Multipler: 5.0
Multipler: 5.1
Multipler: 5.2
Multipler: 5.3
Multipler: 5.4
Multipler: 5.5
Multipler: 5.6
Multipler: 5.7
Multipler: 5.8
Multipler: 5.9
|     |         |   High Movies |   Low movies |   Ratings on High |   Ratings on Low |
|-----|---------|---------------|--------------|-------------------|------------------|
| 4   | 2307.37 |          2231 |        42884 |          21034137 |          4990152 |
| 4.1 | 2365.06 |          2200 |        42915 |          20961671 |          5062618 |
| 4.2 | 2422.74 |          2168 |        42947 |          20884942 |          5139347 |
| 4.3 | 2480.43 |          2117 |        42998 |          20760119 |          5264170 |
| 4.4 | 2538.11 |          2092 |        43023 |          20697404 |          5326885 |
| 4.5 | 2595.8  |          2067 |        43048 |          20633261 |          539102

In [17]:
print(tabulate(results, headers=["High Movies", "Low movies", "Ratings on High", "Ratings on Low"], tablefmt="github"))

|     |         |   High Movies |   Low movies |   Ratings on High |   Ratings on Low |
|-----|---------|---------------|--------------|-------------------|------------------|
| 4   | 2307.37 |          2231 |        42884 |          21034137 |          4990152 |
| 4.1 | 2365.06 |          2200 |        42915 |          20961671 |          5062618 |
| 4.2 | 2422.74 |          2168 |        42947 |          20884942 |          5139347 |
| 4.3 | 2480.43 |          2117 |        42998 |          20760119 |          5264170 |
| 4.4 | 2538.11 |          2092 |        43023 |          20697404 |          5326885 |
| 4.5 | 2595.8  |          2067 |        43048 |          20633261 |          5391028 |
| 4.6 | 2653.48 |          2032 |        43083 |          20541200 |          5483089 |
| 4.7 | 2711.16 |          1998 |        43117 |          20449968 |          5574321 |
| 4.8 | 2768.85 |          1971 |        43144 |          20375950 |          5648339 |
| 4.9 | 2826.53 |          1951 

### Filter ratings based on users

I will find the average ratings per user, and observe how the size of the dataset changes if I keep only the users that have more ratings than a specific threshold

In [19]:
users_ids = set([
    rating["userId"]
    for rating in ratings_json
])

In [20]:
len(users_ids)

270896

In [21]:
ratings_per_user = defaultdict(list)
for rating in ratings_json:
    ratings_per_user[rating["userId"]].append(rating)

In [22]:
ratings_per_user_count = { userId: len(ratings) for userId, ratings in ratings_per_user.items() }

In [23]:
avg_ratings_count = mean(ratings_per_user_count.values())
avg_ratings_count

96.06745393065974

In [24]:
def count_average_ratings_per_movie(kept_ratings):
    ratings_per_movie_count = defaultdict(int)
    for rating in kept_ratings:
        ratings_per_movie_count[rating["movieId"]] += 1
    return mean(ratings_per_movie_count.values())

def count_average_ratings_per_user(kept_ratings):
    ratings_per_user_count = defaultdict(int)
    for rating in kept_ratings:
        ratings_per_user_count[rating["userId"]] += 1
    return mean(ratings_per_user_count.values())

In [25]:
def get_statistics_by_kept_users(kept_users):
    kept_ratings = [
        rating for rating in ratings_json if rating["userId"] in kept_users]
    kept_movies = set([
        rating["movieId"] for rating in kept_ratings])
    avg_ratings_per_movie = count_average_ratings_per_movie(kept_ratings)
    avg_ratings_per_user = count_average_ratings_per_user(kept_ratings)
    return kept_ratings, kept_movies, avg_ratings_per_movie, avg_ratings_per_user

In [26]:
def count_users_ratings_by_threshold(multiplier):
    threshold = avg_ratings_count * multiplier
    very_active_users = set(
        [userId for userId, ratings in ratings_per_user_count.items()
         if ratings >= threshold]
    )
    kept_ratings, kept_movies, avg_ratings_per_movie, avg_ratings_per_user = get_statistics_by_kept_users(very_active_users)
    return [multiplier, threshold, len(very_active_users), len(kept_ratings), len(kept_movies), avg_ratings_per_movie, avg_ratings_per_user]


In [30]:
results = []
for i in tqdm(range(1, 20)):
    results.append(count_users_ratings_by_threshold(i/10))
print("OK")
print(tabulate(results, headers=["Multiplier", "Min user ratings", "Active Users", "Kept Ratings", "Kept Movies", "Avg ratings/movie"], tablefmt="github"))

100%|██████████| 19/19 [05:50<00:00, 18.42s/it]

OK
|   Multiplier |   Min user ratings |   Active Users |   Kept Ratings |   Kept Movies |   Avg ratings/movie |
|--------------|--------------------|----------------|----------------|---------------|---------------------|
|          0.1 |            9.60675 |         233903 |       25847473 |         45090 |             573.242 |
|          0.2 |           19.2135  |         166444 |       24848104 |         45028 |             551.837 |
|          0.3 |           28.8202  |         138664 |       24196963 |         44988 |             537.854 |
|          0.4 |           38.427   |         119264 |       23553402 |         44905 |             524.516 |
|          0.5 |           48.0337  |         105066 |       22939400 |         44853 |             511.435 |
|          0.6 |           57.6405  |          94238 |       22367498 |         44830 |             498.94  |
|          0.7 |           67.2472  |          84828 |       21781353 |         44801 |             486.18  |
|      

In [31]:
results = []
for i in tqdm(range(20, 40)):
    results.append(count_users_ratings_by_threshold(i/10))
print("OK")
print(tabulate(results, headers=["Multiplier", "Min user ratings", "Active Users", "Kept Ratings", "Kept Movies", "Avg ratings/movie"], tablefmt="github"))

100%|██████████| 20/20 [04:24<00:00, 13.24s/it]

OK
|   Multiplier |   Min user ratings |   Active Users |   Kept Ratings |   Kept Movies |   Avg ratings/movie |
|--------------|--------------------|----------------|----------------|---------------|---------------------|
|          2   |            192.135 |          34250 |       15992853 |         44427 |             359.98  |
|          2.1 |            201.742 |          32600 |       15667267 |         44388 |             352.962 |
|          2.2 |            211.348 |          30809 |       15297722 |         44364 |             344.823 |
|          2.3 |            220.955 |          29289 |       14969612 |         44314 |             337.808 |
|          2.4 |            230.562 |          27836 |       14642009 |         44289 |             330.601 |
|          2.5 |            240.169 |          26438 |       14313040 |         44265 |             323.349 |
|          2.6 |            249.775 |          25340 |       14043908 |         44248 |             317.391 |
|      

In [27]:
users_groups = defaultdict(list)

avg_ratings_count = mean(ratings_per_user_count.values())
for userId, ratings_count in ratings_per_user_count.items():
    x = round(ratings_count / avg_ratings_count * 100)
    group = x - x%5
    users_groups[group].append(userId)

In [28]:
len(users_groups)

643

In [29]:
users_groups_count = { key: len(users) for key, users in users_groups.items() }
users_groups_count = { key: users for key, users in sorted(users_groups_count.items(), key=lambda x: x[0]) }

In [30]:
users_groups_count

{0: 14789,
 5: 22204,
 10: 18013,
 15: 44855,
 20: 19451,
 25: 12920,
 30: 10654,
 35: 7128,
 40: 7878,
 45: 6700,
 50: 6393,
 55: 5673,
 60: 4091,
 65: 4463,
 70: 4154,
 75: 3826,
 80: 3379,
 85: 2501,
 90: 2882,
 95: 2569,
 100: 2796,
 105: 3051,
 110: 1946,
 115: 2313,
 120: 2100,
 125: 2006,
 130: 1813,
 135: 1728,
 140: 1331,
 145: 1576,
 150: 1396,
 155: 1452,
 160: 1367,
 165: 999,
 170: 1211,
 175: 1157,
 180: 1114,
 185: 924,
 190: 750,
 195: 911,
 200: 840,
 205: 992,
 210: 954,
 215: 671,
 220: 890,
 225: 796,
 230: 756,
 235: 697,
 240: 585,
 245: 694,
 250: 593,
 255: 624,
 260: 641,
 265: 502,
 270: 544,
 275: 555,
 280: 524,
 285: 462,
 290: 390,
 295: 457,
 300: 434,
 305: 407,
 310: 525,
 315: 355,
 320: 458,
 325: 448,
 330: 344,
 335: 413,
 340: 294,
 345: 350,
 350: 355,
 355: 338,
 360: 340,
 365: 271,
 370: 310,
 375: 296,
 380: 289,
 385: 289,
 390: 219,
 395: 259,
 400: 297,
 405: 223,
 410: 239,
 415: 294,
 420: 193,
 425: 264,
 430: 252,
 435: 219,
 440: 220,


In [33]:
def random_squeeze_group(group, max_users):
    limit = random.randrange(max_users)
    if len(group) <= limit:
        return group
    else:
        return random.sample(group, limit)

def get_statistics_by_groups_kept(min_group, max_group, max_users_per_group, randomize=True):
    groups = [group for group_key, group in users_groups.items() if min_group <= group_key <= max_group and random.random()>0.2]
    users_to_keep = [userId for group in groups for userId in set(random_squeeze_group(group, max_users_per_group))]
    kept_ratings, kept_movies, avg_ratings_per_movie, avg_ratings_per_user = get_statistics_by_kept_users(set(users_to_keep))
    return [len(users_to_keep), len(kept_ratings), len(kept_movies), avg_ratings_per_movie, avg_ratings_per_user, users_to_keep]

In [102]:
results = []
min_groups = [5]
max_groups = [100, 210, 300, 400]
max_users_per_group = [1000, 2000, 3000, 4000, 5000]
for min_group_index in min_groups:
    print("Min group:", min_group_index)
    for max_group_index in max_groups:
        print("Max group:", max_group_index)
        for max_users_per_group_item in max_users_per_group:
            print("Max users per group:", max_users_per_group_item)
            a, b, c, d, users_to_keep = get_statistics_by_groups_kept(min_group_index, max_group_index, max_users_per_group_item)
            results.append([min_group_index, max_group_index, max_users_per_group_item, a, b, c, d])
print(tabulate(results, headers=["Min group", "Max group", "Max allowed users per group", "Kept users", "Kept Ratings", "Kept Movies", "Avg ratings/movie"], tablefmt="github"))


Min group: 5
Max group: 100
Max users per group: 1000
Max users per group: 2000
Max users per group: 3000
Max users per group: 4000
Max users per group: 5000
Max group: 210
Max users per group: 1000
Max users per group: 2000
Max users per group: 3000
Max users per group: 4000
Max users per group: 5000
Max group: 300
Max users per group: 1000
Max users per group: 2000
Max users per group: 3000
Max users per group: 4000
Max users per group: 5000
Max group: 400
Max users per group: 1000
Max users per group: 2000
Max users per group: 3000
Max users per group: 4000
Max users per group: 5000
|   Min group |   Max group |   Max allowed users per group |   Kept users |   Kept Ratings |   Kept Movies |   Avg ratings/movie |
|-------------|-------------|-------------------------------|--------------|----------------|---------------|---------------------|
|           5 |         100 |                          1000 |        20000 |        1044232 |         13211 |             79.0426 |
|          

In [114]:
results = []
min_groups = [5]
max_groups = [300]
max_users_per_group = [200, 300, 500, 1000]
for min_group_index in min_groups:
    print("Min group:", min_group_index)
    for max_group_index in max_groups:
        print("Max group:", max_group_index)
        for max_users_per_group_item in max_users_per_group:
            print("Max users per group:", max_users_per_group_item)
            a, b, c, d, e, users_to_keep = get_statistics_by_groups_kept(min_group_index, max_group_index, max_users_per_group_item)
            results.append([min_group_index, max_group_index, max_users_per_group_item, a, b, c, d, e])
print(tabulate(results, headers=["Min (% avg ratings)", "Max (% avg ratings)", "Max allowed users per group", "Kept users", "Kept Ratings", "Kept Movies", "Avg ratings/movie", "Avg ratings/user"], tablefmt="github"))


Min group: 5
Max group: 300
Max users per group: 200
Max users per group: 300
Max users per group: 500
Max users per group: 1000
|   Min group |   Max group |   Max allowed users per group |   Kept users |   Kept Ratings |   Kept Movies |   Avg ratings/movie |   Avg ratings/user |
|-------------|-------------|-------------------------------|--------------|----------------|---------------|---------------------|--------------------|
|           5 |         300 |                           200 |         4010 |         692954 |         13182 |             52.5682 |            172.806 |
|           5 |         300 |                           300 |         4334 |         628577 |         12519 |             50.2098 |            145.034 |
|           5 |         300 |                           500 |         9882 |        1405211 |         15743 |             89.2594 |            142.199 |
|           5 |         300 |                          1000 |        15750 |        2581925 |         1857

In [41]:
count_average_ratings_per_movie(ratings_json)

576.8433780339134

In [42]:
count_average_ratings_per_user(ratings_json)

96.06745393065974

### Average Dataset with around 2M ratings

In [40]:
# average user has submitted 96 ratings
#  keep users that have ratings_count more than 5% and less than 300% of 96
min_group = 10
max_group = 300
max_users_per_group = 700
# split these users in groups (by 5% distance from average ratings_count)
# keep a random number of users (max 500) from each group 
res = get_statistics_by_groups_kept(min_group, max_group, max_users_per_group)
stats = res[:-1]
users_to_keep = res[-1]
print(tabulate([stats], headers=["Kept users", "Kept Ratings", "Kept Movies", "Avg ratings/movie", "Avg ratings/user"], tablefmt="github"))

|   Kept users |   Kept Ratings |   Kept Movies |   Avg ratings/movie |   Avg ratings/user |
|--------------|----------------|---------------|---------------------|--------------------|
|        15589 |        2481265 |         18516 |             134.007 |            159.168 |


### Conclusion

| graph | users | movies | ratings | avg ratings per movie | avg ratings per user |
| --- | --- | --- | --- | --- | --- |
| small | 700 | 9k | 100k | 11 | 149
| large | 270k | 45k | 25M | 576 | 96 |
| sub-graph | 10k-20k | 10k-20k | 2M | 100-200 | 100-200 |


* To build the sub-graph, I filtered the ratings on the large graph based on the users, following the steps:
    * count the ratings submitted by each user
    * find the average number of ratings per user in the initial graph (96)
    * split the users in groups, based on their ratings count in comparison to the average ratings count per user (96)
    * keep randomly the 80% the groups with users who have submitted from 10 (96 * 10%) to 288 (96 * 300%) ratings
    * keep randomly at most 700 users from each group